### Purpose of this notebook is to balance the classes

In [62]:
%matplotlib inline
import pandas as pd
import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import sys
import sklearn
from sklearn.preprocessing import normalize
import sklearn
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import normalize
sys.path.append('../code')
import helper_functions

### Loading data

In [40]:
# Loading 3 axis data from all sets
x_test, y_test = helper_functions.return_segments_3axis('../data/test.csv', one_hot_encoded = False)
x_train, y_train = helper_functions.return_segments_3axis('../data/train.csv', one_hot_encoded = False)
x_val, y_val = helper_functions.return_segments_3axis('../data/val.csv', one_hot_encoded = False)

# Extracting features
x_train_features = helper_functions.return_features(x_train, axes = 3)
x_val_features = helper_functions.return_features(x_val, axes = 3)
x_test_features = helper_functions.return_features(x_test, axes = 3)

Reducing examples from class 3 since it has maximum number of examples. This technique is known as undersampling and is used to balance datasets.

In [45]:
new_features = list()
new_labels = list()
for features, label in zip(x_train_features, y_train) : 
    if (label == 3) :
        if np.random.random() > 0.5 : 
            new_features.append(features)
            new_labels.append(label)
    else : 
        new_features.append(features)
        new_labels.append(label)

In [66]:
final_features = np.vstack(new_features)
print(final_features.shape)
final_labels = np.vstack(new_labels)
final_labels = final_labels.ravel()
print(final_labels.shape)

(672, 24)
(672,)


In [57]:
x_train_norm = normalize(final_features)
x_test_norm = normalize(x_test_features)
x_val_norm = normalize(x_val_features)

In [74]:
# class_wt = {0 : 3, 1 : 3, 2 : 2, 3 : 1, 4 : 2}
svm_lin = SVC(C = 1e4, class_weight = 'balanced', gamma = 'scale')
svm_lin.fit(x_train_norm, final_labels)
y_pred = svm_lin.predict(x_test_norm)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
y_pred = svm_lin.predict(x_val_norm)
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))
y_pred = svm_lin.predict(x_train_norm)
print(confusion_matrix(final_labels, y_pred))
print(classification_report(final_labels, y_pred))

[[ 3  3  3  3  3]
 [ 2  7  4  4  2]
 [ 8  3  7  1  4]
 [ 9  4  8 10  2]
 [ 3  4  3  3  5]]
              precision    recall  f1-score   support

           0       0.12      0.20      0.15        15
           1       0.33      0.37      0.35        19
           2       0.28      0.30      0.29        23
           3       0.48      0.30      0.37        33
           4       0.31      0.28      0.29        18

    accuracy                           0.30       108
   macro avg       0.30      0.29      0.29       108
weighted avg       0.33      0.30      0.31       108

[[ 2  4  4  4  1]
 [ 1  1  5  3  3]
 [ 5  2  8  2  2]
 [11  7  4  5  8]
 [ 2  6  9  2  5]]
              precision    recall  f1-score   support

           0       0.10      0.13      0.11        15
           1       0.05      0.08      0.06        13
           2       0.27      0.42      0.33        19
           3       0.31      0.14      0.20        35
           4       0.26      0.21      0.23        24

   